In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
sys.path.append(os.path.join(module_path, 'cnn-keras'))

In [2]:
%matplotlib inline

from keras.models import load_model
from keras.optimizers import SGD, Adam, RMSprop
from keras.callbacks import LearningRateScheduler
from keras.utils import np_utils
import keras.backend as K
import numpy as np

import fs
import models
import utils

Using Theano backend.
Using gpu device 0: Quadro K620M (CNMeM is disabled, cuDNN 5004)


In [5]:
DATA_DIR = '../data/imdb-wiki-tiny-dataset'

TRAIN_DATA_OBJ = 'train_data'
VAL_DATA_OBJ = 'val_data'
TEST_DATA_OBJ = 'test_data'

class_names = [
    '[0 - 15]', '[16 - 20]', '[21 - 25]', '[26 - 30]', '[31 - 35]',
    '[36 - 40]', '[41 - 45]', '[46 - 50]', '[51 - 55]', '[56 - 100]'
]

n_classes = 10

batch_size = 30
nb_epoch = 30
learning_rate = 2e-2
decay = 0
l2_reg = 3e-3

lr_boost_epoch = 40
lr_boost_value = 2e-2

X_train = np.load(fs.join(DATA_DIR, TRAIN_DATA_OBJ + '_00.npy')).astype(np.float32)
y_train = np.load(fs.join(DATA_DIR, TRAIN_DATA_OBJ + '_label_age_00.npy')).astype(int)
Y_train = np_utils.to_categorical(y_train, n_classes)

X_val = np.load(fs.join(DATA_DIR, VAL_DATA_OBJ + '.npy')).astype(np.float32)
y_val = np.load(fs.join(DATA_DIR, VAL_DATA_OBJ + '_label_age.npy')).astype(int)
Y_val = np_utils.to_categorical(y_val, n_classes)

X_test = np.load(fs.join(DATA_DIR, TEST_DATA_OBJ + '_00.npy')).astype(np.float32)
y_test = np.load(fs.join(DATA_DIR, TEST_DATA_OBJ + '_label_age_00.npy')).astype(int)
Y_test = np_utils.to_categorical(y_test, n_classes)

def flip(X):
    return X.transpose((0,3,2,1))

def transform(X, mean=None, std=None):
    if std is not None:
        return flip((flip(X) - mean) / (std + 1e-8))
    elif mean is not None:
        return flip(flip(X) - mean)
    else:
        return X

ax = (0,1,2)
mean = flip(X_train).mean(ax)
std = flip(X_train).std(ax)

X_train = transform(X_train, mean, std)
X_val = transform(X_val, mean, std)
X_test = transform(X_test, mean, std)

model = models.get_simple_cnn(input_shape=(3,48,48), n_classes=n_classes, init='glorot_uniform', l2_reg=l2_reg, activation='elu')
opt = SGD(lr=learning_rate, decay=decay)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

def scheduler(epoch):
    if epoch == lr_boost_epoch:
        model.optimizer.lr.set_value(lr_boost_value)
    return float(model.optimizer.lr.get_value())

change_lr = LearningRateScheduler(scheduler)

hist = model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch,
          verbose=1, validation_data=(X_val, Y_val), callbacks=[change_lr])

score = model.evaluate(X_test, Y_test, verbose=0)

print('Test score:', score[0])
print('Test accuracy:', score[1])

Train on 10000 samples, validate on 1000 samples
Epoch 1/30
10000/10000 [==============================] - 62s - loss: 11.2569 - acc: 0.1574 - val_loss: 2.1505 - val_acc: 0.1460
Epoch 2/30
10000/10000 [==============================] - 69s - loss: 10.5078 - acc: 0.1800 - val_loss: 2.1350 - val_acc: 0.1990
Epoch 3/30
10000/10000 [==============================] - 64s - loss: 9.8499 - acc: 0.1872 - val_loss: 2.1349 - val_acc: 0.2050
Epoch 4/30
10000/10000 [==============================] - 65s - loss: 9.2451 - acc: 0.2032 - val_loss: 2.1255 - val_acc: 0.1960
Epoch 5/30
10000/10000 [==============================] - 63s - loss: 8.6907 - acc: 0.2107 - val_loss: 2.1182 - val_acc: 0.1950
Epoch 6/30
10000/10000 [==============================] - 64s - loss: 8.1801 - acc: 0.2113 - val_loss: 2.1164 - val_acc: 0.2020
Epoch 7/30
 4530/10000 [============>.................] - ETA: 32s - loss: 7.8213 - acc: 0.2221

KeyboardInterrupt: 

In [25]:
from sklearn.metrics import confusion_matrix

y_true = y_test
y_pred = model.predict_classes(X_test, batch_size=batch_size, verbose=0)

cm = confusion_matrix(y_true, y_pred)

utils.plot_confusion_matrix(cm, classes=class_names, normalize=True, title='Confusion matrix [test]')

NameError: name 'utils' is not defined